In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

path = "/content/drive/MyDrive/elli"
os.chdir(path)

print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/elli


In [ ]:

!pip install torch-geometric -q

import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from torch_geometric.datasets import EllipticBitcoinDataset
from torch_geometric.nn import SAGEConv, DeepGraphInfomax


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


dataset = EllipticBitcoinDataset(root='./elliptic')
data = dataset[0].to(device)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.1 MB/s eta 0:00:00


Processing...
Done!


In [ ]:
print("Data object structure:", data)
print(f"Number of nodes: {data.num_nodes}, Number of edges: {data.num_edges}, Feature dimension: {data.num_node_features}")

unique, counts = torch.unique(data.y, return_counts=True)
print("Label distribution:")
for label, count in zip(unique.tolist(), counts.tolist()):
    print(f"  Label {label}: {count} nodes")

print(f"Number of training nodes: {data.train_mask.sum().item()}, Number of testing nodes: {data.test_mask.sum().item()}")


Data object structure: Data(x=[203769, 165], edge_index=[2, 234355], y=[203769], train_mask=[203769], test_mask=[203769])
Number of nodes: 203769, Number of edges: 234355, Feature dimension: 165
Label distribution:
  Label 0: 42019 nodes
  Label 1: 4545 nodes
  Label 2: 157205 nodes
Number of training nodes: 29894, Number of testing nodes: 16670


In [ ]:
# Define GraphSAGE model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

model1 = GraphSAGE(data.num_features, 64, 2).to(device)
opt1 = torch.optim.Adam(model1.parameters(), lr=0.01)

# Training function
def train(model, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
@torch.no_grad()
def evaluate(model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train GraphSAGE
for epoch in range(1, 51):
    loss = train(model1, opt1)
    if epoch % 10 == 0:
        acc1 = evaluate(model1)
        print(f'[GraphSAGE] Epoch {epoch} | Loss: {loss:.4f} | Test Acc: {acc1:.4f}')


[GraphSAGE] Epoch 10 | Loss: 0.1742 | Test Acc: 0.9196
[GraphSAGE] Epoch 20 | Loss: 0.1184 | Test Acc: 0.9259
[GraphSAGE] Epoch 30 | Loss: 0.0942 | Test Acc: 0.9314
[GraphSAGE] Epoch 40 | Loss: 0.0785 | Test Acc: 0.9365
[GraphSAGE] Epoch 50 | Loss: 0.0669 | Test Acc: 0.9376


In [ ]:
# Define GraphSAGE model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

model1 = GraphSAGE(data.num_features, 64, 2).to(device)
opt1 = torch.optim.Adam(model1.parameters(), lr=0.01)

# Training function
def train(model, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
@torch.no_grad()
def evaluate(model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train GraphSAGE
for epoch in range(1, 51):
    loss = train(model1, opt1)
    if epoch % 10 == 0:
        acc1 = evaluate(model1)
        print(f'[GraphSAGE] Epoch {epoch} | Loss: {loss:.4f} | Test Acc: {acc1:.4f}')


[GraphSAGE] Epoch 10 | Loss: 0.1789 | Test Acc: 0.9379
[GraphSAGE] Epoch 20 | Loss: 0.1239 | Test Acc: 0.9304
[GraphSAGE] Epoch 30 | Loss: 0.0958 | Test Acc: 0.9374
[GraphSAGE] Epoch 40 | Loss: 0.0790 | Test Acc: 0.9418
[GraphSAGE] Epoch 50 | Loss: 0.0669 | Test Acc: 0.9461


In [ ]:
# Define DGI encoder (SAGE)
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv = SAGEConv(in_channels, hidden_channels)

    def forward(self, x, edge_index):
        return self.conv(x, edge_index)

    def summary(self, z, *args, **kwargs):
        return torch.sigmoid(z.mean(dim=0))

encoder = Encoder(data.num_features, 64).to(device)
dgi = DeepGraphInfomax(
    hidden_channels=64,
    encoder=encoder,
    summary=encoder.summary,
    corruption=lambda x, edge_index: (x[torch.randperm(x.size(0))], edge_index),
).to(device)

opt2 = torch.optim.Adam(dgi.parameters(), lr=0.001)

# Train DGI
for epoch in range(1, 101):
    dgi.train()
    opt2.zero_grad()
    pos_z, neg_z, summary = dgi(data.x, data.edge_index)
    loss = dgi.loss(pos_z, neg_z, summary)
    loss.backward()
    opt2.step()
    if epoch % 20 == 0:
        print(f'[DGI] Epoch {epoch} | Loss: {loss.item():.4f}')

# Use the trained encoder to get z
with torch.no_grad():
    z = encoder(data.x, data.edge_index).detach()

# Define a simple classifier using z
class Classifier(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.lin = torch.nn.Linear(in_dim, out_dim)

    def forward(self, z):
        return self.lin(z)

clf = Classifier(64, 2).to(device)
opt3 = torch.optim.Adam(clf.parameters(), lr=0.01)

# Train classifier
for epoch in range(1, 51):
    clf.train()
    opt3.zero_grad()
    out = clf(z)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    opt3.step()
    if epoch % 10 == 0:
        clf.eval()
        pred = out.argmax(dim=1)
        acc2 = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
        print(f'[DGI+Classifier] Epoch {epoch} | Loss: {loss.item():.4f} | Test Acc: {acc2:.4f}')


[DGI] Epoch 20 | Loss: 1.3332
[DGI] Epoch 40 | Loss: 1.3168
[DGI] Epoch 60 | Loss: 1.3127
[DGI] Epoch 80 | Loss: 1.3079
[DGI] Epoch 100 | Loss: 1.3077
[DGI+Classifier] Epoch 10 | Loss: 0.3885 | Test Acc: 0.4195
[DGI+Classifier] Epoch 20 | Loss: 0.3062 | Test Acc: 0.5254
[DGI+Classifier] Epoch 30 | Loss: 0.2758 | Test Acc: 0.5852
[DGI+Classifier] Epoch 40 | Loss: 0.2596 | Test Acc: 0.6278
[DGI+Classifier] Epoch 50 | Loss: 0.2494 | Test Acc: 0.6611


In [ ]:
from sklearn.metrics import classification_report

# Model 1: GraphSAGE
model1.eval()
out1 = model1(data.x, data.edge_index)
pred1 = out1.argmax(dim=1)
true1 = data.y
mask = data.test_mask

# Model 2: DGI + Classifier
clf.eval()
out2 = clf(z)
pred2 = out2.argmax(dim=1)
true2 = data.y

# Move only the final tensors used in sklearn to CPU
true1_cpu = true1[mask].cpu()
pred1_cpu = pred1[mask].cpu()

true2_cpu = true2[mask].cpu()
pred2_cpu = pred2[mask].cpu()

# Print evaluation metrics for comparison
print("====== GraphSAGE Trained Directly ======")
print(classification_report(true1_cpu, pred1_cpu, digits=4))

print("====== DGI + GraphSAGE Classifier ======")
print(classification_report(true2_cpu, pred2_cpu, digits=4))


====== GraphSAGE Trained Directly ======
              precision    recall  f1-score   support

           0     0.9659    0.9768    0.9713     15587
           1     0.6013    0.5042    0.5485      1083

    accuracy                         0.9461     16670
   macro avg     0.7836    0.7405    0.7599     16670
weighted avg     0.9422    0.9461    0.9439     16670

====== DGI + GraphSAGE Classifier ======
              precision    recall  f1-score   support

           0     0.9743    0.6577    0.7853     15587
           1     0.1322    0.7507    0.2248      1083

    accuracy                         0.6637     16670
   macro avg     0.5533    0.7042    0.5051     16670
weighted avg     0.9196    0.6637    0.7489     16670

